In [53]:
from sklearn import svm
import pymysql
import csv
import math
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stemmer=PorterStemmer()
stop_words = [stemmer.stem(word) for word in stopwords.words('english')]
cvectorizer = CountVectorizer()
tvectorizer = TfidfVectorizer()
unigramTokenizer = TfidfVectorizer(ngram_range=(1,1), stop_words="english", max_features=10000)
bigramTokenizer = TfidfVectorizer(ngram_range=(2,2), stop_words="english", max_features=10000)
trigramTokenizer = TfidfVectorizer(ngram_range=(3,3), stop_words="english", max_features=10000)
combined_features = FeatureUnion([("trigram", trigramTokenizer)])

In [54]:
# data_base=pymysql.connect("localhost","root","Vardhan2@","citizen_complaints_sampled")
# cursor=data_base.cursor()

def getBinary(x):
    if x > 0:
        x = 1
    return x
# query1 = 'SELECT image_count, video_count, url_count, pre4.organization as organization, pre4.tweet_text as tweet_text FROM posts, pre4 where posts.tweet_id=pre4.tweet_id'
# query2 = 'SELECT c_and_g FROM annotated'

# cursor.execute(query2)

# output = [c_and_g[0] for (c_and_g) in cursor]

# cursor.execute(query1)
question_words=defaultdict(lambda: 0)
complaint_words=defaultdict(lambda: 0)
health_words=defaultdict(lambda: 0)

question_words['why']=1
question_words['what']=1
question_words['how']=1
question_words['where']=1
complaint_words['pain']=1
complaint_words['not']=1
def getMatching(s, top_complaint_words):
    words = s.strip().split()
    i = 0
    n = 1
    for word in words:
        word = stemmer.stem(word.lower())
        word = re.sub(r'[^\w\s]','',word)
        if word not in stop_words and word!='':
            n += 1
            if top_complaint_words[word]==1:
                i += 1
    return i/n
def getComplaintScore(s1,s2):
    s=s1+" "+s2
    
    return [getMatching(s,getMatching(s, question_words)), getMatching(s,getMatching(s, complaint_words)), getMatching(s,getMatching(s, health_words))]

In [55]:
file_data = open('DS.csv')
# file_data_list = file_data.split('\n')
file_Reader = csv.reader(file_data)
file_rows = [row for row in file_Reader]
#for row in file_Reader:
#    print(row)
#   break
n={'title':0, 'score':1, 'cmts':2, 'body':3, 'url_count':4, 'is_complaint':5}
file_rows = file_rows[1:]


In [56]:
# tweets=[tweet_text for (image_count, video_count, url_count, organization, tweet_text) in cursor]
posts=[[getComplaintScore(f[n['title']], f[n['body']]), int(f[n['url_count']])] for f in file_rows]

tags = [int(f[n['is_complaint']]) for f in file_rows]

# x=combined_features.fit_transform(tweets)
# x=x.toarray()
# x=x.todense()
# cursor.close()
# data_base.close()
# l1 = []
# for i in range(0,len(media_count)):
#     l2 = []
#     l2.append(media_count[i][0])
#     l2.append(media_count[i][1])
#     y=0
#     if media_count[i][2]=='DelhiPolice':
#         y=1
#     elif media_count[i][2]=='dtptraffic':
#         y=2
#     elif media_count[i][2]=='RailMinIndia':
#         y=3
#     else:
#         y=4
#     l2.append(y)
#     l1.append(l2)
    
# print(l1[0])

# y=np.zeros((len(x),len(x[0])+3))
# for i in range(0,len(x)):
#     for j in range(0,len(x[0])):
#         y[i][j]=x[i][j]
#     k=len(x[0])
#     y[i][k]=media_count[i][0]
#     y[i][k+1]=media_count[i][1]
#     if media_count[i][2]=='DelhiPolice':
#         y[i][k+2]=1
#     elif media_count[i][2]=='dtptraffic':
#         y[i][k+2]=2
#     elif media_count[i][2]=='RailMinIndia':
#         y[i][k+2]=3
#     else:
#         y[i][k+2]=4

TypeError: 'float' object is not subscriptable

In [52]:
# train_data_x1=x[:3*int((x.shape[0])/4)]
# test_data_x1=x[3*int((x.shape[0])/4):]
# train_data_x2=l1[:3*int(len(l1)/4)]
# test_data_x2=l1[3*int(len(l1)/4):]
# train_data_y = output[:3*int(len(output)/4)]
# test_data_y = output[3*int(len(output)/4):]
x=posts
output=tags
train_data_x1=x[:4*int(len(x)/5)]
test_data_x1=x[4*int(len(x)/5):]

# train_data_x2=l1[:4*int(len(l1)/5)]
# test_data_x2=l1[4*int(len(l1)/5):]
train_data_y = output[:4*int(len(x)/5)]
test_data_y = output[4*int(len(x)/5):]

cvm = svm.SVC()
cvm.fit(train_data_x1,train_data_y)
predicted_data_y1 = cvm.predict(test_data_x1)
# cvm.fit(train_data_x2,train_data_y)
# predicted_data_y2 = cvm.predict(test_data_x2)

for i in range(100):
    print(train_data_x1[i], '  ', train_data_y[i])

matched = 0
for i in range(len(test_data_y)):
    if predicted_data_y1[i]==test_data_y[i]:
        matched += 1

print('Accuracy - ', (matched*100)/len(test_data_y))

[2.5649493574615367, 0]    1
[2.6390573296152584, 0]    0
[3.4657359027997265, 0]    0
[2.995732273553991, 0]    0
[2.3978952727983707, 0]    0
[2.8903717578961645, 0]    0
[2.772588722239781, 0]    0
[2.3978952727983707, 0]    0
[2.772588722239781, 0]    0
[2.995732273553991, 0]    0
[2.772588722239781, 0]    0
[3.295836866004329, 0]    0
[3.1354942159291497, 0]    0
[2.6390573296152584, 0]    0
[2.9444389791664403, 0]    1
[4.143134726391533, 2]    1
[2.772588722239781, 0]    0
[2.302585092994046, 0]    0
[3.2188758248682006, 0]    0
[1.791759469228055, 0]    0
[2.6390573296152584, 0]    0
[2.0794415416798357, 0]    0
[5.488937726156687, 0]    1
[2.1972245773362196, 0]    0
[2.70805020110221, 0]    0
[2.1972245773362196, 0]    0
[4.634728988229636, 0]    1
[2.0794415416798357, 0]    0
[2.8903717578961645, 0]    0
[2.833213344056216, 0]    0
[1.9459101490553132, 0]    0
[3.5263605246161616, 0]    0
[2.6390573296152584, 0]    0
[4.762173934797756, 1]    1
[2.4849066497880004, 0]    0
[